In [1]:
import random

import numpy as np
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras import layers, Sequential, Input
from sklearn.model_selection import train_test_split
from generator import Generator
from keras import metrics
import importlib
import inverse
from matplotlib.ticker import PercentFormatter
from inverse import fit_linear

def custom_loss(y_true, y_pred):
    pass
class ConstrainedOutputLayer(tf.keras.layers.Layer):
    def __init__(self, M=None, weight=10, **kwargs):
        super(ConstrainedOutputLayer, self).__init__(**kwargs)
        # Initialize M as None to handle custom initialization in from_config
        if M is not None:
            self.M = tf.constant(M, dtype=tf.float32)
        else:
            self.M = None  # Will be set later during call or from_config
        self.weight = weight

    def call(self, y_out, x_in):
        if self.M is None:
            raise ValueError("Matrix M must be initialized.")
        # Apply the constraint logic
        constraint = tf.matmul(y_out, self.M) - x_in
        self.add_loss(self.weight * tf.reduce_mean(tf.square(constraint)))
        return y_out

    def get_config(self):
        # Get the base config and update it with custom arguments
        config = super(ConstrainedOutputLayer, self).get_config()
        # Serialize M as a list, and weight as is
        config.update({
            'M': self.M.numpy().tolist() if self.M is not None else None,
            'weight': self.weight
        })
        return config

    @classmethod
    def from_config(cls, config):
        # Reconstruct the layer from the config
        M = tf.constant(config['M'], dtype=tf.float32) if config['M'] is not None else None
        weight = config['weight']
        return cls(M=M, weight=weight)


In [244]:
model = tf.keras.models.load_model('data/models/silica_CNN_cut_isotherm_smp2.keras', custom_objects={'abs': tf.math.abs, 'custom_loss': custom_loss,
                                                                                                        'ConstrainedOutputLayer': ConstrainedOutputLayer})

In [245]:
pressures = np.load("data/initial kernels/Pressure_Silica.npy")[:-10]
pore_widths = np.load("data/initial kernels/Size_Kernel_Silica_Adsorption.npy")
kernel = np.load("data/initial kernels/Kernel_Silica_Adsorption.npy")

In [246]:
test_dataset = np.load(f'data/datasets/SMP_test_2.npz')
x_test = test_dataset['x_test']
y_test = test_dataset['y_test']

In [247]:
prediction = model.predict(np.array(x_test))

107/107 [==============================] - 0s 2ms/step


In [219]:
i = random.randint(0, len(x_test))
plt.plot(pore_widths, prediction[i], marker='o', label='Predicted')
plt.plot(pore_widths, y_test[i], marker='o', label='True')
psd = fit_linear(adsorption=x_test[i], kernel=kernel[:, :-10], alpha=0).x
plt.plot(pore_widths, psd, marker='o', label='psd')
plt.gray()
plt.legend()
plt.show()

In [220]:
#i = random.randint(0, len(x_test))
plt.plot(pressures, np.array(x_test[i]), marker='o', label='True')
plt.plot(pressures, np.dot(kernel.T, prediction[i])[:-10], marker='o', label='Predicted')
plt.plot(pressures, np.dot(kernel.T[:-10], y_test[i]), marker='o', label='math')
plt.gray()
plt.legend()
plt.show()
print(i)

672


In [248]:
def calculate_error(isotherm_real, restored_isotherm):
    defined_error = 0
    iso_len = 0
    for i in range(len(restored_isotherm)):
        if isotherm_real[i] != 0:
            defined_error += np.abs((isotherm_real[i] - restored_isotherm[i])/isotherm_real[i])
            iso_len += 1
    if iso_len == 0:
        return 0
    return defined_error / iso_len

def calculate_roughness(psd):
    return np.linalg.norm(np.diff(psd), ord=2)

In [249]:
model2 = tf.keras.models.load_model('data/models/silica_CNN_cut_isotherm.keras', custom_objects={'abs': tf.math.abs, 'custom_loss': custom_loss,
                                                                                                        'ConstrainedOutputLayer': ConstrainedOutputLayer})
prediction2 = model2.predict(np.array(x_test))

107/107 [==============================] - 0s 2ms/step


In [252]:
error_math = []
roughness_math = []

error_lst = []
roughness_lst = []

error_lst2 = []
roughness_lst2 = []

for i in range(len(prediction)):
    restored_isotherm = np.dot(kernel.T[:-10], prediction[i])
    error_lst.append(calculate_error(restored_isotherm, np.array(x_test[i]))*100)
    roughness_lst.append(calculate_roughness(prediction[i]))

    restored_isotherm = np.dot(kernel.T[:-10], prediction2[i])
    error_lst2.append(calculate_error(restored_isotherm, np.array(x_test[i]))*100)
    roughness_lst2.append(calculate_roughness(prediction2[i]))

    restored_isotherm_math = np.dot(kernel.T[:-10], y_test[i])
    error_math.append(calculate_error(restored_isotherm_math, np.array(x_test[i]))*100)
    roughness_math.append(calculate_roughness(y_test[i]))

In [262]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde


x = np.linspace(0, 60, 1000)

kde_error= gaussian_kde(error_lst)
kde_error2= gaussian_kde(error_lst2)
kde_math = gaussian_kde(error_math)

plt.plot(x, kde_error(x), label='Модель на основе экспериментальных изотерм', color='red')
plt.plot(x, kde_error2(x), label='Модель на основе искусственных распределений', color='orange')
#plt.scatter(error_lst, kde_error(error_lst), color='red')

plt.plot(x, kde_math(x), label='Метод NLDFT без регуляризации', color='blue')
#plt.scatter(error_math, kde_math(error_math), color='blue')

plt.legend(loc='upper right')
plt.xlabel("Ошибка описания изотермы %")
plt.title("Распределения ошибок моделей (PDF)")
plt.xlim(8, 60)
plt.grid()
plt.show()

In [256]:
x = np.linspace(0, 100, 1000)

kde_model= gaussian_kde(roughness_lst)
kde_math = gaussian_kde(roughness_math)

plt.plot(x, kde_model(x), label='Roughness model', color='red')
#plt.scatter(roughness_lst, kde_model(roughness_lst), color='red')

plt.plot(x, kde_math(x), label='Roughness math', color='blue')
#plt.scatter(roughness_math, kde_math(roughness_math), color='blue')

plt.legend(loc='upper right')
plt.xlabel("Roughness")
plt.title("Roughness (PDF)")
plt.grid()
plt.show()

In [76]:
error_min = min(error_math)
roughness_min = min(roughness_math)
error_max = max(error_math)
roughness_max = max(roughness_math)


X, Y = np.mgrid[error_min:error_max:300j, roughness_min:roughness_max:300j]
positions = np.vstack([X.ravel(), Y.ravel()])
values = np.vstack([error_math, roughness_math])
kde = gaussian_kde(values)
Z = np.reshape(kde(positions).T, X.shape)

fig, ax = plt.subplots()
ax.imshow(np.rot90(Z), extent=[error_min, error_max, roughness_min, roughness_max], aspect='auto')
ax.plot(error_math, roughness_math, 'k.', markersize=3)
ax.set_xlim([error_min, error_max])
ax.set_ylim([roughness_min, roughness_max])
plt.xlabel("error %")
plt.ylabel("roughness")
plt.show()

In [77]:
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z, edgecolor='royalblue', lw=0.5, rstride=8, cstride=8,
                alpha=0.3, cmap="hsv")
ax.set_xlabel('error %')
ax.set_ylabel('roughness')
plt.show()